# MongoDB Aggregation Pipeline In Python

Learn about the various stages and configurations you can create to configure an Aggregation Pipeline

Stages: [match](#match), [project](#project), [unset](#unset), [limit](#limit), [skip](#skip), [sort](#sort), [count](#count), [sortByCount](#sortByCount), [unwind](#unwind), [group](#group), [addFields](#addFields), [sample](#sample), [lookup](#lookup), [unionWith](#unionWith), [out](#out), [merge](#merge)

Operators: [size](#size-(operator)), [in](#in-(operator)), [arrayElemAt](#arrayElemAt-(operator)), [first](#first-(operator)), [count](#count-(accumulator-operator)), [sum](#sum-(accumulator-operator)), [first, last](#first,-last-(accumulator-operators)), [push](#push-(accumulator-operator)), [addToSet](#addToSet-(accumulator-operator)), [regexMatch](#regexMatch-(operator)), [cond](#cond-(operator)), [Date](#Date-Operators), [expr](#expr-(operator)), [ifNull](#ifNull-(operator)), [type](#type-(operator)), [switch](#switch-(operator))

In [1]:
from pymongo import MongoClient

In [2]:
mongodb_uri = "mongodb://localhost:27017/"
db_name = "aggregation_test"

In [3]:
client = MongoClient(mongodb_uri)
db = client[db_name]

### Helper Function

In [4]:
def print_cursor(cursor):
    for document in cursor:
        print(document, end="\n\n")

### Inserting Some Sample Data

In [5]:
import insert_aggregation_sample_data as iasd
iasd.insert_data(mongodb_uri, db_name)

Entries already exist in the aggregation_test database in the users, products, or orders collection. Insert commands aborted.


### match

In [6]:
match_cursor = db.products.aggregate([
    {"$match": {"name": "Pens"}}
])

In [7]:
print_cursor(match_cursor)

{'_id': ObjectId('61bfb3805121347d31340fd6'), 'name': 'Pens', 'seller_id': ObjectId('61bfb3805121347d31340fcf'), 'tags': ['Office', 'School']}



In [8]:
match_cursor = db.products.aggregate([
    {"$match": {"$or": [{"tags": "Beauty"}, {"tags": "Home"}]}}
])

In [9]:
print_cursor(match_cursor)

{'_id': ObjectId('61bfb3805121347d31340fd4'), 'name': 'Mug', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Home', 'Kitchen']}

{'_id': ObjectId('61bfb3805121347d31340fd5'), 'name': 'Moisturizer', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Beauty']}

{'_id': ObjectId('61bfb3805121347d31340fd7'), 'name': 'Face Cleanser', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Beauty']}

{'_id': ObjectId('61bfb3805121347d31340fd8'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('61bfb3805121347d31340fd1'), 'tags': ['Beauty']}

{'_id': ObjectId('61bfb3805121347d31340fd9'), 'name': 'Eyeliner', 'seller_id': ObjectId('61bfb3805121347d31340fd2'), 'tags': ['Beauty']}



### project

In [10]:
project_cursor = db.products.aggregate([
    {"$project": {"_id": 0,"product_name": "$name", "tags": 1}}
])

In [11]:
print_cursor(project_cursor)

{'tags': ['Home', 'Kitchen'], 'product_name': 'Mug'}

{'tags': ['Beauty'], 'product_name': 'Moisturizer'}

{'tags': ['Office', 'School'], 'product_name': 'Pens'}

{'tags': ['Beauty'], 'product_name': 'Face Cleanser'}

{'tags': ['Beauty'], 'product_name': 'Concealer Makeup'}

{'tags': ['Beauty'], 'product_name': 'Eyeliner'}



In [12]:
match_project_cursor = db.products.aggregate([
    {"$match": {"name": "Pens"}},
    {"$project": {"_id": 0, "product_name": "$name", "tags": 1}}
])

In [13]:
print_cursor(match_project_cursor)

{'tags': ['Office', 'School'], 'product_name': 'Pens'}



### unset

In [14]:
unset_cursor = db.products.aggregate([
    {"$unset": ["_id", "seller_id"]}
])

In [15]:
print_cursor(unset_cursor)

{'name': 'Mug', 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'tags': ['Beauty']}

{'name': 'Pens', 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'tags': ['Beauty']}

{'name': 'Eyeliner', 'tags': ['Beauty']}



In [16]:
unset_cursor = db.products.aggregate([
    {"$match": {"name": "Pens"}},
    {"$unset": ["_id", "seller_id"]}
])

In [17]:
print_cursor(unset_cursor)

{'name': 'Pens', 'tags': ['Office', 'School']}



### limit

In [18]:
limit_cursor = db.products.aggregate([
    {"$limit": 3}
])

In [19]:
print_cursor(limit_cursor)

{'_id': ObjectId('61bfb3805121347d31340fd4'), 'name': 'Mug', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Home', 'Kitchen']}

{'_id': ObjectId('61bfb3805121347d31340fd5'), 'name': 'Moisturizer', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Beauty']}

{'_id': ObjectId('61bfb3805121347d31340fd6'), 'name': 'Pens', 'seller_id': ObjectId('61bfb3805121347d31340fcf'), 'tags': ['Office', 'School']}



### skip

In [20]:
skip_cursor = db.products.aggregate([
    {"$skip": 2}
])

In [21]:
print_cursor(skip_cursor)

{'_id': ObjectId('61bfb3805121347d31340fd6'), 'name': 'Pens', 'seller_id': ObjectId('61bfb3805121347d31340fcf'), 'tags': ['Office', 'School']}

{'_id': ObjectId('61bfb3805121347d31340fd7'), 'name': 'Face Cleanser', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Beauty']}

{'_id': ObjectId('61bfb3805121347d31340fd8'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('61bfb3805121347d31340fd1'), 'tags': ['Beauty']}

{'_id': ObjectId('61bfb3805121347d31340fd9'), 'name': 'Eyeliner', 'seller_id': ObjectId('61bfb3805121347d31340fd2'), 'tags': ['Beauty']}



In [22]:
limit_and_skip_cursor = db.products.aggregate([
    {"$skip": 2},
    {"$limit": 3}
])

In [23]:
print_cursor(limit_and_skip_cursor)

{'_id': ObjectId('61bfb3805121347d31340fd6'), 'name': 'Pens', 'seller_id': ObjectId('61bfb3805121347d31340fcf'), 'tags': ['Office', 'School']}

{'_id': ObjectId('61bfb3805121347d31340fd7'), 'name': 'Face Cleanser', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Beauty']}

{'_id': ObjectId('61bfb3805121347d31340fd8'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('61bfb3805121347d31340fd1'), 'tags': ['Beauty']}



### sort

In [24]:
sort_cursor = db.products.aggregate([
    {"$sort": {"name": 1}}
])

In [25]:
print_cursor(sort_cursor)

{'_id': ObjectId('61bfb3805121347d31340fd8'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('61bfb3805121347d31340fd1'), 'tags': ['Beauty']}

{'_id': ObjectId('61bfb3805121347d31340fd9'), 'name': 'Eyeliner', 'seller_id': ObjectId('61bfb3805121347d31340fd2'), 'tags': ['Beauty']}

{'_id': ObjectId('61bfb3805121347d31340fd7'), 'name': 'Face Cleanser', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Beauty']}

{'_id': ObjectId('61bfb3805121347d31340fd5'), 'name': 'Moisturizer', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Beauty']}

{'_id': ObjectId('61bfb3805121347d31340fd4'), 'name': 'Mug', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Home', 'Kitchen']}

{'_id': ObjectId('61bfb3805121347d31340fd6'), 'name': 'Pens', 'seller_id': ObjectId('61bfb3805121347d31340fcf'), 'tags': ['Office', 'School']}



### count

In [26]:
count_cursor = db.products.aggregate([
    {"$match": {"tags": "Beauty"}},
    {"$count": "beauty_products_count"}
])

In [27]:
print_cursor(count_cursor)

{'beauty_products_count': 4}



### sortByCount

In [28]:
sort_by_count_cursor = db.products.aggregate([
    {"$sortByCount": "$tags"}
])

In [29]:
print_cursor(sort_by_count_cursor)

{'_id': ['Beauty'], 'count': 4}

{'_id': ['Home', 'Kitchen'], 'count': 1}

{'_id': ['Office', 'School'], 'count': 1}



### size (operator)

In [30]:
project_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "name":1, "num_tags": {"$size":"$tags"}, "tags": "$tags"}}
])

In [31]:
print_cursor(project_cursor)

{'name': 'Mug', 'num_tags': 2, 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'num_tags': 1, 'tags': ['Beauty']}

{'name': 'Pens', 'num_tags': 2, 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'num_tags': 1, 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'num_tags': 1, 'tags': ['Beauty']}

{'name': 'Eyeliner', 'num_tags': 1, 'tags': ['Beauty']}



### in (operator)

In [32]:
project_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "name":1, "is_beauty_product": {"$in": ["Beauty", "$tags"]}, "tags": "$tags"}}
])

In [33]:
print_cursor(project_cursor)

{'name': 'Mug', 'is_beauty_product': False, 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'is_beauty_product': True, 'tags': ['Beauty']}

{'name': 'Pens', 'is_beauty_product': False, 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'is_beauty_product': True, 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'is_beauty_product': True, 'tags': ['Beauty']}

{'name': 'Eyeliner', 'is_beauty_product': True, 'tags': ['Beauty']}



### arrayElemAt (operator)

In [34]:
project_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "name":1, "first_tag": {"$arrayElemAt": ["$tags", 0]}, "tags": "$tags"}}
])

In [35]:
print_cursor(project_cursor)

{'name': 'Mug', 'first_tag': 'Home', 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Pens', 'first_tag': 'Office', 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Eyeliner', 'first_tag': 'Beauty', 'tags': ['Beauty']}



### first (operator)

In [36]:
project_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "name":1, "first_tag": {"$first": "$tags"}, "tags": "$tags"}}
])

In [37]:
print_cursor(project_cursor)

{'name': 'Mug', 'first_tag': 'Home', 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Pens', 'first_tag': 'Office', 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Eyeliner', 'first_tag': 'Beauty', 'tags': ['Beauty']}



### unwind

In [38]:
unwind_cursor = db.products.aggregate([
    {"$unwind": "$tags"},
    {"$unset": ["_id", "seller_id"]}
])

In [39]:
print_cursor(unwind_cursor)

{'name': 'Mug', 'tags': 'Home'}

{'name': 'Mug', 'tags': 'Kitchen'}

{'name': 'Moisturizer', 'tags': 'Beauty'}

{'name': 'Pens', 'tags': 'Office'}

{'name': 'Pens', 'tags': 'School'}

{'name': 'Face Cleanser', 'tags': 'Beauty'}

{'name': 'Concealer Makeup', 'tags': 'Beauty'}

{'name': 'Eyeliner', 'tags': 'Beauty'}



In [40]:
unwind_cursor = db.orders.aggregate([
    {"$unwind": "$items"}
])

In [41]:
print_cursor(unwind_cursor)

{'_id': ObjectId('61bfb3805121347d31340fda'), 'items': {'product_id': ObjectId('61bfb3805121347d31340fd5'), 'quantity': 1}}

{'_id': ObjectId('61bfb3805121347d31340fda'), 'items': {'product_id': ObjectId('61bfb3805121347d31340fd7'), 'quantity': 1}}

{'_id': ObjectId('61bfb3805121347d31340fdb'), 'items': {'product_id': ObjectId('61bfb3805121347d31340fd8'), 'quantity': 1}}

{'_id': ObjectId('61bfb3805121347d31340fdb'), 'items': {'product_id': ObjectId('61bfb3805121347d31340fd9'), 'quantity': 1}}

{'_id': ObjectId('61bfb3805121347d31340fdc'), 'items': {'product_id': ObjectId('61bfb3805121347d31340fd6'), 'quantity': 5}}

{'_id': ObjectId('61bfb3805121347d31340fdc'), 'items': {'product_id': ObjectId('61bfb3805121347d31340fd4'), 'quantity': 1}}

{'_id': ObjectId('61bfb3805121347d31340fdd'), 'items': {'product_id': ObjectId('61bfb3805121347d31340fd5'), 'quantity': 2}}

{'_id': ObjectId('61bfb3805121347d31340fdd'), 'items': {'product_id': ObjectId('61bfb3805121347d31340fd9'), 'quantity': 1}}



In [42]:
unwind_cursor = db.products.aggregate([
    {"$match": {"tags": {"$size": 2}}},
    {"$unwind": {"path": "$tags", "includeArrayIndex": "tag_index"}},
    {"$unset": ["_id"]}
])

In [43]:
print_cursor(unwind_cursor)

{'name': 'Mug', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': 'Home', 'tag_index': 0}

{'name': 'Mug', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': 'Kitchen', 'tag_index': 1}

{'name': 'Pens', 'seller_id': ObjectId('61bfb3805121347d31340fcf'), 'tags': 'Office', 'tag_index': 0}

{'name': 'Pens', 'seller_id': ObjectId('61bfb3805121347d31340fcf'), 'tags': 'School', 'tag_index': 1}



### group

In [44]:
group_cursor = db.products.aggregate([
    {"$group": {"_id": "$tags"}}
])

In [45]:
print_cursor(group_cursor)

{'_id': ['Home', 'Kitchen']}

{'_id': ['Beauty']}

{'_id': ['Office', 'School']}



In [46]:
group_cursor = db.products.aggregate([
    {"$unwind": "$tags"},
    {"$group": {"_id": "$tags"}}
])

In [47]:
print_cursor(group_cursor)

{'_id': 'Kitchen'}

{'_id': 'School'}

{'_id': 'Home'}

{'_id': 'Office'}

{'_id': 'Beauty'}



### count (accumulator operator)

In [48]:
group_cursor = db.products.aggregate([
    {"$group": {"_id": "$tags", "num_entries": {"$count": {}}}}
])

In [49]:
print_cursor(group_cursor)

{'_id': ['Home', 'Kitchen'], 'num_entries': 1}

{'_id': ['Beauty'], 'num_entries': 4}

{'_id': ['Office', 'School'], 'num_entries': 1}



### sum (accumulator operator)

In [50]:
group_cursor = db.orders.aggregate([
    {"$unwind": "$items"},
    {"$group": {"_id": "$items.product_id","total_quantity": {"$sum": "$items.quantity"}}}
])

In [51]:
print_cursor(group_cursor)

{'_id': ObjectId('61bfb3805121347d31340fd9'), 'total_quantity': 2}

{'_id': ObjectId('61bfb3805121347d31340fd6'), 'total_quantity': 5}

{'_id': ObjectId('61bfb3805121347d31340fd5'), 'total_quantity': 4}

{'_id': ObjectId('61bfb3805121347d31340fd8'), 'total_quantity': 1}

{'_id': ObjectId('61bfb3805121347d31340fd4'), 'total_quantity': 1}

{'_id': ObjectId('61bfb3805121347d31340fd7'), 'total_quantity': 2}



### first, last (accumulator operators)

In [52]:
group_cursor = db.products.aggregate([
    {"$group": {"_id": "$tags","num_entries": {"$count": {}}, "first": {"$first": "$name"}, "last": {"$last": "$name"}}}
])

In [53]:
print_cursor(group_cursor)

{'_id': ['Home', 'Kitchen'], 'num_entries': 1, 'first': 'Mug', 'last': 'Mug'}

{'_id': ['Beauty'], 'num_entries': 4, 'first': 'Moisturizer', 'last': 'Eyeliner'}

{'_id': ['Office', 'School'], 'num_entries': 1, 'first': 'Pens', 'last': 'Pens'}



### push (accumulator operator)

In [54]:
group_cursor = db.products.aggregate([
    {"$group": {"_id": "$tags", "products": {"$push": "$name"}}}
])

In [55]:
print_cursor(group_cursor)

{'_id': ['Home', 'Kitchen'], 'products': ['Mug']}

{'_id': ['Beauty'], 'products': ['Moisturizer', 'Face Cleanser', 'Concealer Makeup', 'Eyeliner']}

{'_id': ['Office', 'School'], 'products': ['Pens']}



### addToSet (accumulator operator)

In [56]:
group_cursor = db.products.aggregate([
    {"$group": {"_id": "$tags", "products": {"$addToSet": "$name"}}}
])

In [57]:
print_cursor(group_cursor)

{'_id': ['Home', 'Kitchen'], 'products': ['Mug']}

{'_id': ['Beauty'], 'products': ['Face Cleanser', 'Moisturizer', 'Concealer Makeup', 'Eyeliner']}

{'_id': ['Office', 'School'], 'products': ['Pens']}



In [58]:
group_cursor = db.products.aggregate([
    {"$group": {"_id": "$tags", "products": {"$addToSet": {"name":"$name","seller_id": "$seller_id"}}}}
])

In [59]:
print_cursor(group_cursor)

{'_id': ['Home', 'Kitchen'], 'products': [{'name': 'Mug', 'seller_id': ObjectId('61bfb3805121347d31340fce')}]}

{'_id': ['Beauty'], 'products': [{'name': 'Face Cleanser', 'seller_id': ObjectId('61bfb3805121347d31340fce')}, {'name': 'Concealer Makeup', 'seller_id': ObjectId('61bfb3805121347d31340fd1')}, {'name': 'Eyeliner', 'seller_id': ObjectId('61bfb3805121347d31340fd2')}, {'name': 'Moisturizer', 'seller_id': ObjectId('61bfb3805121347d31340fce')}]}

{'_id': ['Office', 'School'], 'products': [{'name': 'Pens', 'seller_id': ObjectId('61bfb3805121347d31340fcf')}]}



### $$ROOT (system variable)

In [60]:
group_cursor = db.products.aggregate([
    {"$group": {"_id": "$tags", "products": {"$addToSet": "$$ROOT"}}}
])

In [61]:
print_cursor(group_cursor)

{'_id': ['Home', 'Kitchen'], 'products': [{'_id': ObjectId('61bfb3805121347d31340fd4'), 'name': 'Mug', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Home', 'Kitchen']}]}

{'_id': ['Beauty'], 'products': [{'_id': ObjectId('61bfb3805121347d31340fd8'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('61bfb3805121347d31340fd1'), 'tags': ['Beauty']}, {'_id': ObjectId('61bfb3805121347d31340fd9'), 'name': 'Eyeliner', 'seller_id': ObjectId('61bfb3805121347d31340fd2'), 'tags': ['Beauty']}, {'_id': ObjectId('61bfb3805121347d31340fd5'), 'name': 'Moisturizer', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Beauty']}, {'_id': ObjectId('61bfb3805121347d31340fd7'), 'name': 'Face Cleanser', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Beauty']}]}

{'_id': ['Office', 'School'], 'products': [{'_id': ObjectId('61bfb3805121347d31340fd6'), 'name': 'Pens', 'seller_id': ObjectId('61bfb3805121347d31340fcf'), 'tags': ['Office', 'School']}]}



### addFields

In [62]:
add_fields_cursor = db.products.aggregate([
    {"$match": {"name": "Pens"}},
    {"$addFields": {"my_new_field": "hi there", "num_tags": {"$size": "$tags"}}}
])

In [63]:
print_cursor(add_fields_cursor)

{'_id': ObjectId('61bfb3805121347d31340fd6'), 'name': 'Pens', 'seller_id': ObjectId('61bfb3805121347d31340fcf'), 'tags': ['Office', 'School'], 'my_new_field': 'hi there', 'num_tags': 2}



### sample

In [64]:
sample_cursor = db.products.aggregate([
    {"$sample": {"size": 3}},
    {"$unset": ["_id", "seller_id"]}
])

In [65]:
print_cursor(sample_cursor)

{'name': 'Moisturizer', 'tags': ['Beauty']}

{'name': 'Mug', 'tags': ['Home', 'Kitchen']}

{'name': 'Concealer Makeup', 'tags': ['Beauty']}



### lookup

In [66]:
lookup_cursor = db.products.aggregate([
    {"$lookup": {
        "from": "users",
        "localField": "seller_id",
        "foreignField": "_id",
        "as": "sellers"
    }} 
])

In [67]:
print_cursor(lookup_cursor)

{'_id': ObjectId('61bfb3805121347d31340fd4'), 'name': 'Mug', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Home', 'Kitchen'], 'sellers': [{'_id': ObjectId('61bfb3805121347d31340fce'), 'name': 'Sarah'}]}

{'_id': ObjectId('61bfb3805121347d31340fd5'), 'name': 'Moisturizer', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Beauty'], 'sellers': [{'_id': ObjectId('61bfb3805121347d31340fce'), 'name': 'Sarah'}]}

{'_id': ObjectId('61bfb3805121347d31340fd6'), 'name': 'Pens', 'seller_id': ObjectId('61bfb3805121347d31340fcf'), 'tags': ['Office', 'School'], 'sellers': [{'_id': ObjectId('61bfb3805121347d31340fcf'), 'name': 'Bob', 'premium_exp_date': datetime.datetime(2021, 12, 28, 10, 0)}]}

{'_id': ObjectId('61bfb3805121347d31340fd7'), 'name': 'Face Cleanser', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Beauty'], 'sellers': [{'_id': ObjectId('61bfb3805121347d31340fce'), 'name': 'Sarah'}]}

{'_id': ObjectId('61bfb3805121347d31340fd8'), 'name': 'Concealer

In [68]:
lookup_cursor = db.products.aggregate([
    {"$lookup": 
        {"from": "users",
         "localField": "seller_id",
         "foreignField": "_id",
         "as": "sellers"
    }},
    {"$project": {"_id": 0, "product_name": "$name", "seller_name": {"$first": "$sellers.name"}}}
])

In [69]:
print_cursor(lookup_cursor)

{'product_name': 'Mug', 'seller_name': 'Sarah'}

{'product_name': 'Moisturizer', 'seller_name': 'Sarah'}

{'product_name': 'Pens', 'seller_name': 'Bob'}

{'product_name': 'Face Cleanser', 'seller_name': 'Sarah'}

{'product_name': 'Concealer Makeup', 'seller_name': 'Lisa'}

{'product_name': 'Eyeliner', 'seller_name': 'Jessica'}



In [70]:
lookup_cursor = db.users.aggregate([
    {"$lookup": 
        {"from": "products",
         "localField": "_id",
         "foreignField": "seller_id",
         "as": "products"
    }},
    {"$addFields": {"num_products": {"$size": "$products"}}},
    {"$match": {"num_products": {"$gte": 1}}},
    {"$project": {"_id": 0, "seller_name": "$name", "products": "$products.name"}},
])

In [71]:
print_cursor(lookup_cursor)

{'seller_name': 'Sarah', 'products': ['Mug', 'Moisturizer', 'Face Cleanser']}

{'seller_name': 'Bob', 'products': ['Pens']}

{'seller_name': 'Lisa', 'products': ['Concealer Makeup']}

{'seller_name': 'Jessica', 'products': ['Eyeliner']}



In [72]:
lookup_cursor = db.products.aggregate([
    {"$group": {"_id": "$seller_id", "product_names": {"$push": "$name"}}},
    {"$lookup": 
        {"from": "users",
         "localField": "_id",
         "foreignField": "_id",
         "as": "sellers"
    }},
    {"$project": {"_id": 0, "seller_name": {"$first": "$sellers.name"}, "products": "$product_names"}},
])

In [73]:
print_cursor(lookup_cursor)

{'seller_name': 'Sarah', 'products': ['Mug', 'Moisturizer', 'Face Cleanser']}

{'seller_name': 'Jessica', 'products': ['Eyeliner']}

{'seller_name': 'Bob', 'products': ['Pens']}

{'seller_name': 'Lisa', 'products': ['Concealer Makeup']}



### unionWith

In [74]:
union_cursor = db.products.aggregate([
    {"$unionWith": "users"}
])

In [75]:
print_cursor(union_cursor)

{'_id': ObjectId('61bfb3805121347d31340fd4'), 'name': 'Mug', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Home', 'Kitchen']}

{'_id': ObjectId('61bfb3805121347d31340fd5'), 'name': 'Moisturizer', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Beauty']}

{'_id': ObjectId('61bfb3805121347d31340fd6'), 'name': 'Pens', 'seller_id': ObjectId('61bfb3805121347d31340fcf'), 'tags': ['Office', 'School']}

{'_id': ObjectId('61bfb3805121347d31340fd7'), 'name': 'Face Cleanser', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Beauty']}

{'_id': ObjectId('61bfb3805121347d31340fd8'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('61bfb3805121347d31340fd1'), 'tags': ['Beauty']}

{'_id': ObjectId('61bfb3805121347d31340fd9'), 'name': 'Eyeliner', 'seller_id': ObjectId('61bfb3805121347d31340fd2'), 'tags': ['Beauty']}

{'_id': ObjectId('61bfb3805121347d31340fce'), 'name': 'Sarah'}

{'_id': ObjectId('61bfb3805121347d31340fcf'), 'name': 'Bob', 'premium_exp_date': d

### regexMatch (operator)

In [76]:
search_input = "IS"

In [77]:
union_cursor = db.products.aggregate([
    {"$unionWith": "users"},
#     {"$addFields": {"matched": { '$regexMatch': { "input": "$name", "regex": search_input}}}},
    {"$addFields": {"matched": { '$regexMatch': { "input": "$name", "regex": search_input, "options":"i" }}}},
    {"$match": {"matched": True}},
    {"$unset": ["matched"]}
])

In [78]:
print_cursor(union_cursor)

{'_id': ObjectId('61bfb3805121347d31340fd5'), 'name': 'Moisturizer', 'seller_id': ObjectId('61bfb3805121347d31340fce'), 'tags': ['Beauty']}

{'_id': ObjectId('61bfb3805121347d31340fd1'), 'name': 'Lisa'}



### out
*Note: You can potentially overwrite all your data in a collection with this stage, use with caution*

In [79]:
out_cursor = db.products.aggregate([
    {"$match": {"tags": "Beauty"}},
    {"$out": {"db": "aggregation_test","coll": "beauty_products"}}
])

In [80]:
# The "$out" stage will not return cursor result entries since they were written to the specified collection
print_cursor(out_cursor)

### merge
*Note: You can potentially overwrite data within a collection with this stage, use with caution*

In [81]:
merge_cursor = db.products.aggregate([
    {"$match": {"tags": "Beauty"}},
#     {"$project": {"test": "value"}},
#     {"$merge": {"into": {"db": "aggregation_test","coll": "beauty_products"}}}
    {"$merge": {
        "into": {"db": "aggregation_test","coll": "beauty_products"},
        "on": "_id", # must specify a indexed attribute
        "whenMatched": "replace",
        "whenNotMatched":  "insert"
   }}
])

In [82]:
# The "$merge" stage will not return cursor result entries since they were used to modify the specified collection
print_cursor(merge_cursor)

### cond (operator)

In [83]:
conditional_cursor = db.users.aggregate([
    {"$project": {"_id": 0, "name": 1, "is_bob": {
        "$cond": {"if": {"$eq": ["$name", "Bob"]}, "then": True, "else": False}
    }}}
])

In [84]:
print_cursor(conditional_cursor)

{'name': 'Sarah', 'is_bob': False}

{'name': 'Bob', 'is_bob': True}

{'name': 'Jose', 'is_bob': False}

{'name': 'Lisa', 'is_bob': False}

{'name': 'Jessica', 'is_bob': False}

{'name': 'Tina', 'is_bob': False}

{'is_bob': False}



In [85]:
conditional_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "name": 1, "tags": 1, "is_beauty_product": {
        "$cond": {"if": {"$in": ["Beauty", "$tags"]}, "then": True, "else": False}
    }}}
])

In [86]:
print_cursor(conditional_cursor)

{'name': 'Mug', 'tags': ['Home', 'Kitchen'], 'is_beauty_product': False}

{'name': 'Moisturizer', 'tags': ['Beauty'], 'is_beauty_product': True}

{'name': 'Pens', 'tags': ['Office', 'School'], 'is_beauty_product': False}

{'name': 'Face Cleanser', 'tags': ['Beauty'], 'is_beauty_product': True}

{'name': 'Concealer Makeup', 'tags': ['Beauty'], 'is_beauty_product': True}

{'name': 'Eyeliner', 'tags': ['Beauty'], 'is_beauty_product': True}



### $$NOW (system variable)

In [87]:
now_cursor = db.users.aggregate([
    {"$match": {"name": "Bob"}},
    {"$project": {"_id": 0, "name": 1, "date_joined": "$$NOW"}}
])

In [88]:
print_cursor(now_cursor)

{'name': 'Bob', 'date_joined': datetime.datetime(2021, 12, 19, 23, 50, 2, 50000)}



### Date Operators

In [89]:
date_add_cursor = db.users.aggregate([
    {"$match": {"name": "Bob"}},
    {"$addFields": {"date_joined": "$$NOW"}},
    {"$project": {"_id": 0, "name": 1, "date_joined": 1, "premium_exp_date": {
        "$dateAdd": {"startDate": "$date_joined", "unit": "day", "amount": 7}
    }}}
])

In [90]:
print_cursor(date_add_cursor)

{'name': 'Bob', 'date_joined': datetime.datetime(2021, 12, 19, 23, 50, 2, 82000), 'premium_exp_date': datetime.datetime(2021, 12, 26, 23, 50, 2, 82000)}



In [91]:
date_diff_cursor = db.users.aggregate([
    {"$match": {"name": "Bob"}},
    {"$addFields": {"date_joined": "$$NOW"}},
    {"$project": {"_id": 0, "name": 1, "date_joined": 1, "premium_exp_date": {
        "$dateAdd": {"startDate": "$date_joined", "unit": "week", "amount": 2}
    }}},
    {"$addFields": {"premium_days_left": {
        "$dateDiff": {"startDate": "$date_joined","endDate": "$premium_exp_date", "unit": "day"}
    }}},
])

In [92]:
print_cursor(date_diff_cursor)

{'name': 'Bob', 'date_joined': datetime.datetime(2021, 12, 19, 23, 50, 2, 114000), 'premium_exp_date': datetime.datetime(2022, 1, 2, 23, 50, 2, 114000), 'premium_days_left': 14}



In [93]:
date_to_parts_cursor = db.users.aggregate([
    {"$match": {"name": "Bob"}},
    {"$addFields": {"date_joined": "$$NOW"}},
    {"$project": {"_id": 0, "name": 1, "date_joined": 1, "date_joined_parts": {
        "$dateToParts": {"date": "$date_joined"}
    }}}
])

In [94]:
print_cursor(date_to_parts_cursor)

{'name': 'Bob', 'date_joined': datetime.datetime(2021, 12, 19, 23, 50, 2, 144000), 'date_joined_parts': {'year': 2021, 'month': 12, 'day': 19, 'hour': 23, 'minute': 50, 'second': 2, 'millisecond': 144}}



In [95]:
date_year_cursor = db.users.aggregate([
    {"$match": {"name": "Bob"}},
    {"$project": {"_id": 0, "name": 1, "date_joined": 1, "date_joined_year": {
        "$year": "$$NOW"
    }}}
])

In [96]:
print_cursor(date_year_cursor)

{'name': 'Bob', 'date_joined_year': 2021}



### expr (operator)

In [97]:
# Without Expressions
add_field_cursor = db.users.aggregate([
    {"$match": {"name": "Bob"}},
    {"$addFields": {"date_joined": "$$NOW"}},
    {"$addFields": {"premium_exp_date": {
        "$dateAdd": {"startDate": "$date_joined", "unit": "day", "amount": 7}
    }}},
    {"$addFields": {"premium_days_left": {
        "$dateDiff": {"startDate": "$date_joined","endDate": "$premium_exp_date", "unit": "day"}
    }}},
    {"$match": {"premium_days_left": {"$lt": 10}}},
])

In [98]:
print_cursor(add_field_cursor)

{'_id': ObjectId('61bfb3805121347d31340fcf'), 'name': 'Bob', 'premium_exp_date': datetime.datetime(2021, 12, 26, 23, 50, 2, 206000), 'date_joined': datetime.datetime(2021, 12, 19, 23, 50, 2, 206000), 'premium_days_left': 7}



In [99]:
# With Expressions
expresion_cursor = db.users.aggregate([
    {"$match": {"name": "Bob"}},
    {"$addFields": {"date_joined": "$$NOW"}},
    {"$addFields": {"premium_exp_date": {
        "$dateAdd": {"startDate": "$date_joined", "unit": "day", "amount": 7}
    }}},
    {"$match": {"$expr": {
        "$lt": [{"$dateDiff": {"startDate": "$date_joined","endDate": "$premium_exp_date", "unit": "day"}}, 10] 
    }}},
])

In [100]:
print_cursor(expresion_cursor)

{'_id': ObjectId('61bfb3805121347d31340fcf'), 'name': 'Bob', 'premium_exp_date': datetime.datetime(2021, 12, 26, 23, 50, 2, 237000), 'date_joined': datetime.datetime(2021, 12, 19, 23, 50, 2, 237000)}



### ifNull (operator)

In [101]:
### A user with no name was added, and Bob was given a 'premium_exp_date' in the compass app
if_null_cursor = db.users.aggregate([
#     {"$project": {"_id": 0, "name": 1, "premium_exp_date": 1}},
    {"$project": {"_id": 0, "name": 1, "has_premium_exp_date": {
        "$ifNull": ["$premium_exp_date", "$name", False]
    }}},
])

In [102]:
print_cursor(if_null_cursor)

{'name': 'Sarah', 'has_premium_exp_date': 'Sarah'}

{'name': 'Bob', 'has_premium_exp_date': datetime.datetime(2021, 12, 28, 10, 0)}

{'name': 'Jose', 'has_premium_exp_date': 'Jose'}

{'name': 'Lisa', 'has_premium_exp_date': 'Lisa'}

{'name': 'Jessica', 'has_premium_exp_date': 'Jessica'}

{'name': 'Tina', 'has_premium_exp_date': 'Tina'}

{'has_premium_exp_date': False}



### type (operator)

In [103]:
type_cursor = db.users.aggregate([
    {"$project": {"_id": 0, "name": 1, "name_type": {"$type": "$name"}, "premium_exp_date_type": {"$type": "$premium_exp_date"},
                 "_id_type": {"$type": "$_id"}
                 }},
])

In [104]:
print_cursor(type_cursor)

{'name': 'Sarah', 'name_type': 'string', 'premium_exp_date_type': 'missing', '_id_type': 'objectId'}

{'name': 'Bob', 'name_type': 'string', 'premium_exp_date_type': 'date', '_id_type': 'objectId'}

{'name': 'Jose', 'name_type': 'string', 'premium_exp_date_type': 'missing', '_id_type': 'objectId'}

{'name': 'Lisa', 'name_type': 'string', 'premium_exp_date_type': 'missing', '_id_type': 'objectId'}

{'name': 'Jessica', 'name_type': 'string', 'premium_exp_date_type': 'missing', '_id_type': 'objectId'}

{'name': 'Tina', 'name_type': 'string', 'premium_exp_date_type': 'missing', '_id_type': 'objectId'}

{'name_type': 'missing', 'premium_exp_date_type': 'missing', '_id_type': 'objectId'}



### switch (operator)

In [105]:
def func_1():
    print("This is Func 1")
    
def func_2():
    print("This is Func 2")
    
def func_3():
    print("This is Func 3")

In [106]:
my_input_val = "d"

if my_input_val == "a":
    func_1()
elif my_input_val == "b":
    func_2()
elif my_input_val == "c":
    func_3()
else:
    func_1()

This is Func 1


In [112]:
### Sarah was given a 'premium_exp_date' that was earlier than today's date in the compass app
switch_cursor = db.users.aggregate([
    {"$project": {"_id": 0, "name": 1, "user_account_error": {
        "$switch": {
            "branches": [
                {"case": {"$eq": [{"$type": "$name"}, 'missing']}, "then": "Missing Name"},
                {"case": {"$eq": [{"$type": "$premium_exp_date"}, 'missing']}, "then": "Missing Premium Expiration Date"},
                {"case": {"$lt": ["$premium_exp_date", "$$NOW"]}, "then": "Premium Membership Expired"}],
            "default": "No Errors Found"
        }
    }}},
])

In [113]:
print_cursor(switch_cursor)

{'name': 'Sarah', 'user_account_error': 'Premium Membership Expired'}

{'name': 'Bob', 'user_account_error': 'No Errors Found'}

{'name': 'Jose', 'user_account_error': 'Missing Premium Expiration Date'}

{'name': 'Lisa', 'user_account_error': 'Missing Premium Expiration Date'}

{'name': 'Jessica', 'user_account_error': 'Missing Premium Expiration Date'}

{'name': 'Tina', 'user_account_error': 'Missing Premium Expiration Date'}



In [114]:
### Rolling back the changes we made to Bob, Sarah, and the user with no name.
db.users.update_many({}, {"$unset": {"premium_exp_date": 1}})
db.users.delete_many({"name": {"$exists": False}})